In [ ]:
import os
import pickle
import jsonlines
import pandas as pd
import numpy as np
import json
import copy
from tqdm import tqdm

In [ ]:
prompt_template = "The point of interest has following attributes: \n name is <NAME>; category is <CATEGORY>; type is <TYPE>; open status is <OPEN>; review count is <COUNT>; city is <CITY>; average score is <STARS>."

In [ ]:
data = json.load(open("./handled/item2attributes.json", "r"))

In [ ]:
data["PzOqRohWw7F7YEPBz6AubA"]

In [ ]:
item_data = {}
for key, value in tqdm(data.items()):
    item_str = copy.deepcopy(prompt_template)
    item_str = item_str.replace("<NAME>", value["name"])
    cate_str = ""
    for cate in value["categories"]:
        cate_str += (cate + " ")
    item_str = item_str.replace("<CATEGORY>", cate_str)
    item_str = item_str.replace("<TYPE>", value["type"])
    item_str = item_str.replace("<OPEN>", str(value["open"]))
    item_str = item_str.replace("<COUNT>", str(value["review_count"]))
    item_str = item_str.replace("<CITY>", value["city"])
    item_str = item_str.replace("<STARS>", str(value["stars"]))
    
    item_data[key] = item_str

In [ ]:
item_data['PzOqRohWw7F7YEPBz6AubA']

In [ ]:
json.dump(item_data, open("./handled/item_str.json", "w"))

In [ ]:
item_data = json.load(open("./handled/item_str.json", "r"))

In [ ]:
import jsonlines

def save_data(data_path, data):
    '''write all_data list to a new jsonl'''
    with jsonlines.open("./handled/"+ data_path, "w") as w:
        for meta_data in data:
            w.write(meta_data)

id_map = json.load(open("./handled/id_map.json", "r"))["item2id"]
json_data = []
for key, value in item_data.items():
    json_data.append({"input": value, "target": "", "item": key, "item_id": id_map[key]})

save_data("item_str.jsonline", json_data)

In [ ]:
import requests
import json

url = ""

payload = json.dumps({
   "model": "text-embedding-ada-002",
   "input": "The food was delicious and the waiter..."
})
headers = {
   'Authorization': '',
   'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
   'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

In [ ]:
re_json = json.loads(response.text)

In [ ]:
len(re_json["data"][0]["embedding"])

In [ ]:
def get_response(prompt):
    url = ""

    payload = json.dumps({
    "model": "text-embedding-ada-002",
    "input": prompt
    })
    headers = {
    'Authorization': '',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    re_json = json.loads(response.text)

    return re_json["data"][0]["embedding"]

In [ ]:
item_emb = {}

In [ ]:
while 1:    # avoid broken due to internet connection
    try:
        for key, value in tqdm(item_data.items()):
            if key not in item_emb.keys():
                item_emb[key] = get_response(value)
    except:
        continue
    if len(item_emb) == len(item_data):
        break

In [ ]:
len(item_emb)

In [ ]:
id_map = json.load(open("./handled/id_map.json", "r"))["id2item"]
emb_list = []
for id in range(1, len(item_emb)+1):
    meta_emb = item_emb[id_map[str(id)]]
    emb_list.append(meta_emb)

emb_list = np.array(emb_list)
pickle.dump(emb_list, open("./handled/itm_emb_np.pkl", "wb"))